In [17]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv("../data/profiles.csv")

In [5]:
def create_dummy(df):
    """
    Creates dummy variables for all the categorial variables in a DataFrame.
    Input: pandas DataFrame
    Output: pandas DataFrame
    """
    cat_cols = df.select_dtypes(include="object").columns
    num_cols = df.select_dtypes(exclude="object").columns
    dummy_df = pd.get_dummies(df[cat_cols],
                              prefix=cat_cols,
                              prefix_sep="_",
                              columns=cat_cols)
                              
    df_new = pd.concat([df[num_cols], dummy_df], axis=1)
    return df_new

In [20]:
df_no_essays = df.drop([f"essay{n}" for n in np.arange(0,10)], axis=1)
df_dummies = create_dummy(df_no_essays)
df_dummies.head()

,age,height,income,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,body_type_full figured,body_type_jacked,...,"speaks_english, yiddish (poorly), spanish (okay), latin (poorly)","speaks_english, yiddish (poorly), spanish (poorly)","speaks_french (fluently), german (fluently), spanish (fluently), english","speaks_french (okay), english","speaks_portuguese (fluently), italian (okay), english (fluently), spanish (fluently)",status_available,status_married,status_seeing someone,status_single,status_unknown
0,22,75.0,-1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,35,70.0,80000,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,38,68.0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,23,71.0,20000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,29,66.0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
